In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pylab
import os
import pandas as pd
import seaborn as sns
import pytraj as pt
import mdtraj as mdt
from pathlib import Path
import json
from pytraj import matrix
import Bio
from Bio.Seq import Seq
from Bio.PDB import *
import nglview as nv
import urllib

path='/Users/alecloftus/Desktop/PKHLAB/PV_Reengagement/Workflow'
os.chdir(path)
print(os.getcwd())
print(path)
print('Hello')

/Users/alecloftus/Desktop/PKHLAB/PV_Reengagement/Workflow
/Users/alecloftus/Desktop/PKHLAB/PV_Reengagement/Workflow
Hello


In [3]:
#download PDB file

def download_pdb(pdbcode, datadir, downloadurl="https://files.rcsb.org/download/"):
    """
    Downloads a PDB file from the Internet and saves it in a data directory.
    :param pdbcode: The standard PDB ID e.g. '3ICB' or '3icb'
    :param datadir: The directory where the downloaded file will be saved
    :param downloadurl: The base PDB download URL, cf.
        `https://www.rcsb.org/pages/download/http#structures` for details
    :return: the full path to the downloaded PDB file or None if something went wrong
    """
    pdbfn = pdbcode + ".pdb"
    url = downloadurl + pdbfn
    outfnm = os.path.join(datadir, pdbfn)
    #print(outfnm)
    try:
        urllib.request.urlretrieve(url, outfnm)
        return outfnm
    except Exception as err:
        return 'ERROR'
'''
for pdb in pdb_list:
    pdb_file=download_pdb(pdb,path)
#print(pdb_file)
'''
#get PDB structure information
parser = PDBParser(PERMISSIVE=1)
pdb_file=path + '/1rwy.pdb'
structure_id='1rwy'
structure = parser.get_structure(structure_id,pdb_file)
model=structure[0]
chain=model["A"]
print(chain)
residue=chain[1]
atom=residue["CA"]

res_list = Selection.unfold_entities(structure, "R")


#BUILD PEPTIDE FOR FASTA SEQUENCE
from Bio.PDB.Polypeptide import PPBuilder
model_nr = 1
ppb = PPBuilder()
for pp in ppb.build_peptides(chain,aa_only=True):
    print(pp)
    #seq = sequence of peptide, aka our FASTA
    seq = pp.get_sequence()
    print(seq)
    seq_length=len(seq)


structure_resolution = structure.header["resolution"]
#print(structure_resolution)
#structure_keywords = structure.header["keywords"]
#print(structure_keywords)

class ChainSelect(Select):
    def accept_chain(self, chain):
        if chain== model["A"]:
            return True
        else:
            return False

io = PDBIO()
io.set_structure(chain)
io.save("1RWY_chainA.pdb", ChainSelect())


<Chain id=A>
<Polypeptide start=1 end=109>
SMTDLLSAEDIKKAIGAFTAADSFDHKKFFQMVGLKKKSADDVKKVFHILDKDKSGFIEEDELGSILKGFSSDARDLSAKETKTLMAAGDKDGDGKIGVEEFSTLVAES


In [142]:
#lists of one-letter code by charge
positively_charged = ['H','K','R']
negatively_charged = ['D','E']
polar_uncharged = ['S','T','N','Q']
hydrophobic = ['A','V','I','L','M','F','Y','W']
special = ['C','G','P']

#GOF for polar uncharged are the polar-charged
GOF = {'A':['S','T','N','Q'],'C':['S','T','N','Q'],'D':['H','K','R'],'E':['H','K','R'],'F':['S','T','N','Q'],'G':['S','T','N','Q'],
       'H':['S','T','N','Q'],'I':['S','T','N','Q'],'K':['D','E'],'L':['S','T','N','Q'],'M':['S','T','N','Q'],'N':['D','E','H','K','R'],
       'P':['S','T','N','Q'],'Q':['D','E','H','K','R'],'R':['D','E'],'S':['D','E','H','K','R'],'T':['D','E','H','K','R'],
       'V':['S','T','N','Q'],'W':['S','T','N','Q'],'Y':['S','T','N','Q']}

LOF = {'A':['D','E','H','K','R'],'C':['D','E','H','K','R'],'D':['A','V','I','L','M','F','Y','W'],'E':['A','V','I','L','M','F','Y','W'],
       'F':['D','E','H','K','R'],'G':['D','E','H','K','R'],'H':['A','V','I','L','M','F','Y','W'],'I':['D','E','H','K','R'],
       'K':['A','V','I','L','M','F','Y','W'],'L':['D','E','H','K','R'],'M':['D','E','H','K','R'],'N':['C','G','P'],'P':['D','E','H','K','R'],
       'Q':['C','G','P'],'R':['A','V','I','L','M','F','Y','W'],'S':['C','G','P'],'T':['C','G','P'],'V':['D','E','H','K','R'],
       'W':['D','E','H','K','R'],'Y':['D','E','H','K','R']}

In [ ]:
#function that's a bunch of if/else for determining contact types
'''
def functionType(pairs,GOF,LOF):
    for key in pairs:
        if (key in pairs and value == pairs[key]) ==True:
        for i in range(len(pairs[key])):
            
    return
'''

In [92]:
#identify interfacial pairs
#contacts

trajectory="1rwy.pdb"
topology="1rwy.pdb"
traj= pt.load(trajectory,top=topology)
print(traj)
ref = traj[0]
system = structure_id
'''
pt.info("nativecontacts")
#command = 'nativecontacts name NC1 :1-:36 :37-:109 resout mindist maxdist distance 13.0'
#command = 'nativecontacts name NC1 @CA= @CA= byresidue out ' + system + '_contacts.dat mindist maxdist distance 13.0 resout ' \
#        + system + '_resout.dat writecontacts ' + system + '_contacts_byAtm.dat series seriesout ' + system + '_nativeSeries.dat'
#command = 'nativecontacts name NC1 @CA= @CA= byresidue out distance 13.0 resout '+ system + '_resout.dat '
command = 'nativecontacts name NC1 @CA= @CA= byresidue writecontacts ' + system + '_contacts_byAtm.dat'
print(command)
#contact_data = pt.compute(command, traj)

outfile_name=system + '_outfile.txt'

file = open(outfile_name,'w')
file.write(str(contact_data))
file.close()
#contact_data = pt.native_contacts(traj, mask=':1-:36', mask2=':37-:109',ref=ref, writecontacts=outfile, distance=8.0)
print(contact_data)
#print(contact_data.keys())
print(contact_data.values())
'''
#Domain cutoffs can be determined down the workflow to be less specific to aPV 1RWY
distance_cutoff=9.0
domain_cutoff=36
next_domain=domain_cutoff+1
contacts=[]
distances=[]
pairs={}
keys=[]
for i in range(1,seq_length+1):
    for j in range(1,seq_length+1):
        mask=':'+str(i) + '@CA :' + str(j) + '@CA'
        #print(mask)
        distance = pt.distance(traj,mask=mask)
        #print(distance[0])
        naming=':' +str(i) + ' :' + str(j) + ' ' + str(distance)
        #distances.append('Res' + str(i) + ' - Res' + str(j) + '= ' + str(distance))
        #Currently, the cross-domain cutoffs are determined here, but can be moved down in workflow if want pairing to include everything
        if distance <= distance_cutoff and distance >= 3.0 and (j>i+4 or j<i-4):# and i<next_domain and j>domain_cutoff:
            #contacts.append(mask)
            distances.append(naming)
            if i in pairs:
                pairs[i].append(j)
            else:
                pairs[i] = [j]
#print(contacts)
#print(distances)
print(pairs)


#plt.plot(contact_data)

pytraj.Trajectory, 1 frames: 
Size: 0.000037 (GB)
<Topology: 1660 atoms, 111 residues, 3 mols, non-PBC>
           
{1: [6, 46, 49], 2: [7, 8, 45, 46, 49, 69, 70, 71], 3: [8, 11, 70, 71], 5: [38, 42, 43, 45, 46], 6: [1, 11, 33, 34, 35, 38, 46, 70], 7: [2, 12, 33, 34], 8: [2, 3, 13, 33, 70, 71], 9: [14, 33], 10: [15, 32, 33, 34], 11: [3, 6, 16, 32, 33, 34, 70, 71], 12: [7, 17, 33, 71], 13: [8, 32, 33], 14: [9, 28, 29, 30, 31, 32, 33, 34], 15: [10, 29, 32, 33, 67, 70, 71, 73, 74, 75], 16: [11, 74], 17: [12, 23, 24, 29, 32], 18: [23, 24, 25, 29], 19: [24, 74, 75], 20: [75], 21: [75, 76, 77, 78, 80, 81], 22: [77, 78, 79, 80, 81], 23: [17, 18, 78, 81], 24: [17, 18, 19, 29, 77, 78, 81, 85], 25: [18, 30, 81, 85], 26: [31, 85, 88, 105, 106, 109], 27: [32, 105, 106, 108, 109], 28: [14, 33, 109], 29: [14, 15, 17, 18, 24, 34, 35], 30: [14, 25, 35, 36, 106, 109], 31: [14, 26, 36, 37, 106, 109], 32: [10, 11, 13, 14, 15, 17, 27], 33: [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 28, 70], 34: [6, 7, 10, 11, 1

In [138]:
#pull interfacing residues from list of contacts, converting contact numbers to res letter codes
#print(pairs)
#print(pairs.keys())
#print(seq)
code_dict={}
mut_pairs={}
GoF={}
LoF={}
#print(pairs)
array_of_code_contacts=[]
not_contacting=[]
for i in range(1,len(seq)+1):
    if i not in pairs.keys():
        not_contacting.append(i)
print(not_contacting)

for key in pairs:
    key_letter=seq[key-1]
    #print(key_letter)
    #print(pairs[key])
    '''
    if key_letter in polar_uncharged:
        GoF[key_letter] = 
    elif key_letter in positively_charged:

    elif key_letter in negatively_charged:

    elif key_letter in hydrophobic:

    
    for i in range(len(pairs[key])):
        residue_from_value = pairs[key][i]-1
        #print(str(residue_from_value) + ' ' + str(seq[residue_from_value]))
        
    
    #print(key)
    #print(pairs[key])
    key_letter=seq[key-1]
    #print(key_letter)
    '''
    list=[key,key_letter]
    for i in range(len(pairs[key])):
        #print(pairs[key][i])
        #print(i)
        residue_from_value = pairs[key][i]-1
        list.append(seq[residue_from_value])
        #print(str(key-1) + ' ' + str(residue_from_value) + ' ' + str(seq[residue_from_value]))
        
        '''
        if key_letter in code_dict:
          code_dict[key_letter].append(seq[residue_from_value])
        elif key_letter not in code_dict:
            code_dict[key_letter] = [seq[residue_from_value]]
        '''
        
    array_of_code_contacts.append(list)
    #print(seq[key-1])

#get each residue in the pair down to their one-letter code
print(array_of_code_contacts)
print(len(array_of_code_contacts))
#print(code_dict)

#make a dictionary of the contacts?

[4, 55, 94]
[[1, 'S', 'L', 'V', 'I'], [2, 'M', 'S', 'A', 'K', 'V', 'I', 'G', 'F', 'S'], [3, 'T', 'A', 'I', 'F', 'S'], [5, 'L', 'K', 'D', 'V', 'K', 'V'], [6, 'L', 'S', 'I', 'V', 'G', 'L', 'K', 'V', 'F'], [7, 'S', 'M', 'K', 'V', 'G'], [8, 'A', 'M', 'T', 'K', 'V', 'F', 'S'], [9, 'E', 'A', 'V'], [10, 'D', 'I', 'M', 'V', 'G'], [11, 'I', 'T', 'L', 'G', 'M', 'V', 'G', 'F', 'S'], [12, 'K', 'S', 'A', 'V', 'S'], [13, 'K', 'A', 'M', 'V'], [14, 'A', 'E', 'K', 'F', 'F', 'Q', 'M', 'V', 'G'], [15, 'I', 'D', 'F', 'M', 'V', 'L', 'F', 'S', 'D', 'A', 'R'], [16, 'G', 'I', 'A'], [17, 'A', 'K', 'S', 'F', 'F', 'M'], [18, 'F', 'S', 'F', 'D', 'F'], [19, 'T', 'F', 'A', 'R'], [20, 'A', 'R'], [21, 'A', 'R', 'D', 'L', 'S', 'K', 'E'], [22, 'D', 'L', 'S', 'A', 'K', 'E'], [23, 'S', 'A', 'F', 'S', 'E'], [24, 'F', 'A', 'F', 'T', 'F', 'L', 'S', 'E', 'L'], [25, 'D', 'F', 'F', 'E', 'L'], [26, 'H', 'Q', 'L', 'A', 'L', 'V', 'S'], [27, 'K', 'M', 'L', 'V', 'E', 'S'], [28, 'K', 'A', 'V', 'S'], [29, 'F', 'A', 'I', 'A', 'F', 'F'

In [141]:
import random
GOF_pairings = {}
LOF_pairings = {}
#generate GOF and LOF mutations based on found interfacing contacts
for i in range(len(array_of_code_contacts)):
    key_resnumber = array_of_code_contacts[i][0]
    WT_key = array_of_code_contacts[i][1]
    random_GOF_mut = random.choice(GOF[WT_key])
    
    GOF_pairings[key_resnumber]= [WT_key,random_GOF_mut]
    '''
    if key_resnumber in GOF_pairings:
        GOF_pairings[key_resnumber].append(WT_key)
        GOF_pairings[key_resnumber].append(random_GOF_mut)
    elif key_letter not in code_dict:
    '''
    
    random_LOF_mut = random.choice(LOF[WT_key])
    LOF_pairings[key_resnumber] = [WT_key,random_LOF_mut]

#print(GOF_pairings)
#print(LOF_pairings)


random_select = random.sample(GOF_pairings.items(),50) + random.sample(LOF_pairings.items(),50)

print(random_select)
print(len(random_select))

[(77, ['L', 'Q']), (68, ['K', 'E']), (35, ['L', 'T']), (75, ['R', 'E']), (97, ['I', 'N']), (49, ['I', 'S']), (51, ['D', 'H']), (95, ['G', 'S']), (78, ['S', 'R']), (100, ['E', 'R']), (12, ['K', 'D']), (26, ['H', 'S']), (18, ['F', 'Q']), (79, ['A', 'N']), (96, ['K', 'D']), (5, ['L', 'S']), (69, ['G', 'N']), (48, ['H', 'Q']), (92, ['D', 'R']), (42, ['D', 'K']), (29, ['F', 'Q']), (30, ['F', 'Q']), (84, ['T', 'R']), (37, ['K', 'E']), (9, ['E', 'K']), (41, ['D', 'K']), (109, ['S', 'D']), (10, ['D', 'H']), (91, ['K', 'E']), (88, ['A', 'Q']), (105, ['L', 'Q']), (54, ['K', 'D']), (89, ['G', 'S']), (33, ['V', 'Q']), (56, ['G', 'T']), (70, ['F', 'N']), (24, ['F', 'T']), (13, ['K', 'E']), (47, ['F', 'N']), (14, ['A', 'T']), (59, ['E', 'H']), (58, ['I', 'S']), (66, ['I', 'S']), (27, ['K', 'D']), (74, ['A', 'T']), (106, ['V', 'N']), (52, ['K', 'D']), (34, ['G', 'S']), (87, ['A', 'S']), (63, ['L', 'Q']), (57, ['F', 'R'])]


/var/folders/tp/lkm8lbb14gb6_ysd0kvxpkgm0000gn/T/ipykernel_54703/1989072474.py:25: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_select = random.sample(GOF_pairings.items(),50) + random.sample(LOF_pairings.items(),50)


In [ ]:
#pyROSETTA

#Python
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *
from pyrosetta.toolbox import *

#Core Includes
#from rosetta.core.kinematics import MoveMap
from pyrosetta.rosetta.core.kinematics import FoldTree
from rosetta.core.pack.task import TaskFactory
from rosetta.core.pack.task import operation
from rosetta.core.simple_metrics import metrics
from rosetta.core.select import residue_selector as selections
from rosetta.core import select
from rosetta.core.select.movemap import *

#Protocol Includes
from rosetta.protocols import minimization_packing as pack_min
from rosetta.protocols import relax as rel
from rosetta.protocols.antibody.residue_selector import CDRResidueSelector
from rosetta.protocols.antibody import *
from rosetta.protocols.loops import *
from rosetta.protocols.relax import FastRelax


init('-use_input_sc -input_ab_scheme AHo_Scheme -ignore_unrecognized_res \
     -ignore_zero_occupancy false -load_PDB_components false -relax:default_repeats 2 -no_fconfig')

pose=pose_from_pdb(pdb_file)
#relax the structure
testPose = Pose()
testPose.assign(pose)
print(testPose)

relax = FastRelax()
scorefxn = get_fa_scorefxn()
relax.set_scorefxn(scorefxn)
relax = rosetta.protocols.relax.FastRelax()
relax.constrain_relax_to_start_coords(True)
print(relax)

#FUNCTION FOR MUTATION

def pack(pose, posi, amino, scorefxn):

    # Select Mutate Position
    mut_posi = pyrosetta.rosetta.core.select.residue_selector.ResidueIndexSelector()
    mut_posi.set_index(posi)
    #print(pyrosetta.rosetta.core.select.get_residues_from_subset(mut_posi.apply(pose)))

    # Select Neighbor Position
    nbr_selector = pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector()
    nbr_selector.set_focus_selector(mut_posi)
    nbr_selector.set_include_focus_in_subset(True)
    #print(pyrosetta.rosetta.core.select.get_residues_from_subset(nbr_selector.apply(pose)))

    # Select No Design Area
    not_design = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(mut_posi)
    #print(pyrosetta.rosetta.core.select.get_residues_from_subset(not_design.apply(pose)))

    # The task factory accepts all the task operations
    tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

    # These are pretty standard
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())

    # Disable Packing
    prevent_repacking_rlt = pyrosetta.rosetta.core.pack.task.operation.PreventRepackingRLT()
    #True indicates here that we are flipping the selection.  So that we are turning off everything but the CDR and its neighbors.
    prevent_subset_repacking = pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(prevent_repacking_rlt, nbr_selector, True )
    tf.push_back(prevent_subset_repacking)

    # Disable design
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
        pyrosetta.rosetta.core.pack.task.operation.RestrictToRepackingRLT(),not_design))

    # Enable design
    aa_to_design = pyrosetta.rosetta.core.pack.task.operation.RestrictAbsentCanonicalAASRLT()
    aa_to_design.aas_to_keep(amino)
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(aa_to_design, mut_posi))
    
    # Create Packer
    packer = pyrosetta.rosetta.protocols.minimization_packing.PackRotamersMover()
    packer.task_factory(tf)
    print(tf.create_task_and_apply_taskoperations(pose))

    #Perform The Move
    if not os.getenv("DEBUG"):
      packer.apply(pose)

for pairing in random_select:
    res_number = pairing[0]
    WT_res = pairing[1][0]
    mut_res = pairing[1][1]
    mutation_name=structure_id + '_' + WT_res + str(res_number) + mut_res + '.pdb'
    print(mutation_name)
    #Load the previously-relaxed pose for mutation
    relaxPose = pose_from_pdb(pdb_file)
    print(relaxPose.sequence())
    #Clone it
    original = relaxPose.clone()
    scorefxn = get_score_function()
    print("\nOld Energy:", scorefxn(original),"\n")
    pack(relaxPose, res_number, mut_res, scorefxn)
    #pack(relaxPose, 81, 'D', scorefxn)
    print("\nNew Energy:", scorefxn(relaxPose),"\n")
    
    #relaxPose.dump_pdb(mutation_name)
    
    print(relaxPose.sequence())
    
    #Set relaxPose back to original
    relaxPose = original.clone()

#Figure out how to minimize the structure or select the rotamers of residues in the region that produces the smallest energy/strain

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2024 [Rosetta PyRosetta4.Release.python310.mac 2024.42+release.3366cf78a3df04339d1982e94531b77b098ddb99 2024-10-11T08:24:04] retrieved from: http://www.pyrosetta.org
core.init: Rosetta version: PyRosetta4.Release.python310.mac r388 2024.42+release.3366cf78a3 3366cf78a3df04339d1982e94531b77b098ddb99 http://www.pyrosetta.org 2024-10-11T08:2

In [ ]:
# WT/RES_NUMBER/MUT    FASTA    OLD ENERGY    NEW ENERGY    